In [235]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tt
import experiments_stuff
from bilinear_package.src import primitives
from bilinear_package.src.hadamard_product import generalizedApproximateHadamardProduct
from bilinear_package.src.convolution import approximateConvolution, preciseConvolution
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
def create_exponential_grid(n, d, A, alpha=1):
    x = np.reshape(np.linspace(-A, A, d, endpoint=True), (1, d, 1))
    ones =  np.reshape(np.ones(d), (1, d, 1))
    base = [ones for _ in range(n)]
    X = []
    for i in range(n):
        kek = base.copy()
        kek[i] = x.copy()
        X.append(tt.vector.from_list(kek))
    c = tt.multifuncrs2(X, lambda x: np.exp(-alpha * np.sqrt(np.sum(x * x, axis = 1))), eps=1E-6, verb=0)
    return tt.vector.to_list(c)

In [188]:
def create_invertable_grid(n, d, A, alpha=1):
    x = np.reshape(np.linspace(-A, A, d, endpoint=True), (1, d, 1))
    ones =  np.reshape(np.ones(d), (1, d, 1))
    base = [ones for _ in range(n)]
    X = []
    for i in range(n):
        kek = base.copy()
        kek[i] = x.copy()
        X.append(tt.vector.from_list(kek))
    c = tt.multifuncrs2(X, lambda x: 1 / np.sqrt(np.sum(x * x, axis = 1)), eps=1E-6, verb=0)
    return tt.vector.to_list(c)

In [189]:
tensor = create_exponential_grid(2, 6, 1)

In [190]:
tt.vector.from_list(tensor)

This is a 2-dimensional tensor 
r(0)=1, n(0)=6 
r(1)=6, n(1)=6 
r(2)=1 

In [191]:
def create_2d_exponential_grid(d, A, alpha=1):
    x = np.linspace(-A, A, d, endpoint=True)
    kek = x[:, None] * x[:, None] + x[None, :] * x[None, :] 
    return np.exp(-alpha * np.sqrt(kek))

In [192]:
def create_2d_invertable_grid(d, A, alpha=1):
    x = np.linspace(-A, A, d, endpoint=True)
    kek = x[:, None] * x[:, None] + x[None, :] * x[None, :] 
    return 1 / np.sqrt(kek)

In [239]:
a = create_exponential_grid(6, 30, 116, alpha=0.1)
tt.vector.from_list(a)

This is a 6-dimensional tensor 
r(0)=1, n(0)=30 
r(1)=14, n(1)=30 
r(2)=14, n(2)=30 
r(3)=14, n(3)=30 
r(4)=14, n(4)=30 
r(5)=14, n(5)=30 
r(6)=1 

In [242]:
b = create_invertable_grid(6, 30, 116)
tt.vector.from_list(b)

This is a 6-dimensional tensor 
r(0)=1, n(0)=30 
r(1)=14, n(1)=30 
r(2)=15, n(2)=30 
r(3)=15, n(3)=30 
r(4)=15, n(4)=30 
r(5)=14, n(5)=30 
r(6)=1 

In [244]:
bench = time.time()
for i in range(20):
    approx = approximateConvolution(a, b, desired_ranks=np.ones(5, dtype=int) * 6, seed=255)
print(time.time() - bench)

3.0013227462768555


In [245]:
precise = preciseConvolution(a, b)

In [246]:
bench = time.time()
for i in range(20):
    tt.vector.from_list(precise).round(0, 6)
print(time.time() - bench)

/home/divanik/.local/lib/python3.8/site-packages/tt/core/vector.py:412: ComplexWarning: Casting complex values to real discards the imaginary part
  _tt_f90.tt_f90.dtt_compr2(c.n, c.r, c.ps, self.core, eps, rmax)


17.25714111328125


In [247]:
primitives.ttTensorsUnsymmetricalRelativeComparance(approx, precise)

(0.00033241089159884775+0j)